In [1]:
# 使用 py-redis 连接到 Redis 数据库
!pip install -q redis

You should consider upgrading via the '/Users/sanhehu/.pyenv/versions/3.8.11/bin/python3.8 -m pip install --upgrade pip' command.


## Bitmap 数据结构

**什么是 Bitmap**

Bitmap (位图) 是一种支持位操作的数据结构. 常用于为海量对象储存 boolean 信息. 例如 1M 个 用户, 随便给一个用户检查今天是否登录过. 普通编程语言中常用的替代数据结构是 Set (集合), 但是 Set 的存储效率要比 bitmap 高得多. 检索效率上两者一致. 但是 Bitmap 能提供更多的功能, 例如统计有多少人登录过.

我们看一个例子就可以了:

假设有 8 个用户, UserId 分别是 ``uid-1, uid-2, ..., uid-8`` 用 Set 存储, 每次用户登录, 就将 UserId 添加到这个 Set 中. 在内存中我们要保存这些 key 的 hash. 如果这些 hash 的空间是 128 位的 MD5, 那么每一个元素的 Key 就要占用至少 128 bit 的空间, 更别提额外的用于维护 hash 表的存储空间了.

而如果用 Bitmap, 我们只需要一个 8 位的空间 ``[i=1,v=0][i=2,v=0][i=3,v=0][i=4,v=0][i=5,v=0][i=6,v=0][i=7,v=0][i=8,v=0]`` 这里 i 指的是 index 位置序号, v 指的是值. 我们给每个用户的 UserId 从 1 开始编号. 如果 1 号用户登录了, 就把 i=1 的那个 bit 位置设为 1 即可.

**Bitmap 在 Redis 中的应用**

Bitmap 在 Redis 中的本质是一个 ASCII 字符串. 只不过支持高性能的位操作.

比如说你对一个 Key (所有的操作都是对一个 Key 所对应的内存地址进行的操作, 下面就不说省略了), 的第 2 位设为 1. 由于最小字符串占用 8 bit, 那么 Redis 就会开辟一个 8 bit 的空间. ``0000-0010`` (第一位在最右边). 对于没有定义的位, 默认值是 0. 换一个例子, 如果你设第 37 位为 1, 那么 Redis 就会开辟一个 40 bit 的空间 (8 的倍数).

Redis 中的最大字符串的大小是 256 MB. 也就是 256 * 1024 * 1024 * 8 = 2,147,483,648 个位空间, 可以表示 21.47 亿个不同元素的状态.


参考资料:

- bitmap 相关命令: https://redis.io/commands/?group=bitmap&name=bit



In [143]:
# 获得 Python 的版本信息
import sys
print(sys.version_info)

sys.version_info(major=3, minor=8, micro=11, releaselevel='final', serial=0)


In [144]:
# 连接到 AWS ElasticCache Redis Cluster
import redis

endpoint = "sanhe-dev.hozbo8.ng.0001.use1.cache.amazonaws.com"
r = redis.Redis(host=endpoint, port=6379, db=0)

In [145]:
# 做一些简单的测试
print(r.set("test", "alice"))
print(r.get("test"))

True
b'alice'


**例1. 位, 整数, 字符串的相互转换**

下面的实验中, 我们开辟一个 8 位的 bitmap 空间. 每一位都设为 1. 那么对应的整数应该是 ``2**8 - 1 = 255``. 由于 255 无法对应 ASCII 中的字符串, 但是却可以对应 Python 中的 bytes 数据结构.

In [146]:
# 设定该实验使用的默认 Key
KEY = "bitmap-test"

# 清除之前的结果
r.delete(KEY)

1

In [147]:
# 开辟一个 8 位的 Bitmap, 每位都设为 1
for offset in range(8):
    r.setbit(KEY, offset, 1)

In [148]:
# 对每个 offset (Redis 里叫 offset, 和 index 是一个意思, 都是从 0 开始的, 用 0 表示第一个位置)
for offset in range(8):
    v = r.getbit(KEY, offset)
    print(f"value at offset {offset} = {v}")

value at offset 0 = 1
value at offset 1 = 1
value at offset 2 = 1
value at offset 3 = 1
value at offset 4 = 1
value at offset 5 = 1
value at offset 6 = 1
value at offset 7 = 1


In [149]:
# Bitmap 从存储的角度看就是一个 bytes string
# b'\xff' 的意思是只有一个 bytes 相当于 8 位. f 是 16 进制中 0123456789abcdef 中的 15. 
# 一个 f 相当于 4 个二进制的 1 (1111). 那么 255 就 相当于 ff.
# 如果是 16 位的数看起来就是 b'\xff\xff', 以此类推, 每 8 位都要用 '\x' 作为前缀表示
r.get(KEY)

b'\xff'

In [150]:
# Python ord 内置函数可以把 bytes string 转化为整数
# 1 位全 1 等于 2**1 - 1 = 1, 2 位全 1 等于 2**2 - 1 = 3. 8 位全 1 等于 2**8 - 1 = 255
ord(r.get(KEY))

255

**例2. 用字符串对 bitmap 空间进行批量初始化**

在例1 中我们是一位一位的设置值的, 每一次都是一次 API 操作, 效率不高. 而由于 bitmap 的本质就是一个字符串, 那么我们可以对其进行批量设置. 但是我们要学会如何用字符串来表示 bitmap 空间的方法. 在 Python 中 int.to_bytes 就是这样的一个函数, 他可以将整数转化成 bytes string.

In [151]:
r.delete(KEY)

# 例如我们想要 8 位全 1, 那么这个二进制数就是 255.
# 这里 length 是指总共有几个 bytes, 一个 bytes 是 8 个比特. 
# 255 是 2 的 8 次方意味着只要 8 个比特即可表示, 那么 8 / 8 = length = 1 即可.
# python 中 byteorder 如果是 16 位以下的数用 little, 以上用 big
r.set(KEY, (2**8-1).to_bytes(length=1, byteorder="big"))

True

In [152]:
r.get(KEY)

b'\xff'

In [153]:
ord(r.get(KEY))

255

**例3. 任意int, bytes 相互转化**

In [154]:
2 ** 128 - 1

340282366920938463463374607431768211455

In [155]:
# int -> bytes
(2 ** 128 - 1).to_bytes(int(128/8), byteorder='big')

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'

In [156]:
# bytes -> int
int.from_bytes(b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff', byteorder="big")

340282366920938463463374607431768211455

## Bitmap 实际应用

我们有 256 个任务编号一直从 0 ~ 255. 这些任务可能成功也可能失败, 用 0 表示失败, 1 表示成功. 我们用 Dynamodb 数据库 和 位图 两种方案来 track 状态, 看看 位图 有什么优势.

**Dynamodb 方案**

我们在 Dynamodb 创建 256 条 item, key 分别是 0 ~ 255, value 可能是 0 和 1. 默认情况下所有的 value 都是 0. 一旦任务成功我们就把对应的 key 的值设为 1.

**Redis Bitmap 方案**

我们在 Redis 创建一个占用 256 bit 的位图空间, 一个字符串需要 8 bit, 总共需要 256 / 8 = 32 个字符. 相当于一个长度为 32 的字符串. 


In [196]:
import math
import random


# 设置该实验使用的默认的 Key
KEY = "task-status-tracking"

# 设置不同的元素的个数
n_task = 100

# 计算需要的 bitmap 的位空间大小
n_bit_in_bitmap = math.ceil(n_task / 8) * 8

# 将最高位设为 0 就会自动将所有低位设为 0.
# 有的人会想要将所有的位设置为 1. 但这是没有必要的. 
# 因为实际业务中, 通常用 0 表示任务的默认状态, 1 表示特殊状态. 你需要对特殊状态的数量进行统计
# 如果你的业务刚好相反, 你完全可以逻辑再做一次反转, 然后通过计算获得结果
def init():
    print("初始化 bitmap, 把所有 task 的状态设为未完成.")
    r.delete(KEY)
    r.setbit(KEY, n_bit_in_bitmap-1, 0)


def run_task(task_id: int):
    success = random.randint(1, 100) >= 30
    if success:
        r.setbit(KEY, task_id, 1)


def run_all_task():
    print("运行所有任务, 任务以 70% 的概率成功.")
    for task_id in range(n_task):
        run_task(task_id)

In [203]:
# 对单个任务进行测试
init()

task_id = 7
r.setbit(KEY, task_id, 1)
print(f"task_id = {task_id} 的任务成功了!")
print(f"总共有 {r.bitcount(KEY)} 任务成功了.")

# task_id = 4
# r.setbit(KEY, task_id, 1)
# print(f"task_id = {task_id} 的任务成功了!")
# print(f"总共有 {r.bitcount(KEY)} 任务成功了.")

初始化 bitmap, 把所有 task 的状态设为未完成.
task_id = 7 的任务成功了!
总共有 1 任务成功了.


In [204]:
# 将所有的位设为 1 的方法
for i in r.get(KEY):
    print(i)

1
0
0
0
0
0
0
0
0
0
0
0
0


In [181]:
# 运行所有任务
init()
run_all_task()
print(f"总共有 {r.bitcount(KEY)} 任务成功了.")

初始化 bitmap, 把所有 task 的状态设为未完成.
运行所有任务, 任务以 70% 的概率成功.
总共有 67 任务成功了.


In [ ]:
1000